<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Test-a-first-naive-bayesian-model" data-toc-modified-id="Test-a-first-naive-bayesian-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test a first naive bayesian model</a></span></li><li><span><a href="#RF" data-toc-modified-id="RF-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>RF</a></span></li><li><span><a href="#XGBOOST" data-toc-modified-id="XGBOOST-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>XGBOOST</a></span></li><li><span><a href="#XGBOOST-FORCE" data-toc-modified-id="XGBOOST-FORCE-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>XGBOOST FORCE</a></span></li></ul></div>

## Packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.simplefilter('ignore')
from utils.utils import time_difference_good_format
import pandas as pd
import time
import os
import psutil

## Test a first naive bayesian model

In [ ]:
from models.naive_bayes_k import naive_bayes_k

In [ ]:
k_list = []
begin_time = time.time()
for k in range(1,6):
    all_results = []
    for taxonomy_level in range(7):
        for selected_primer in ['V1','V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V12', 'V23', 'V34', 'V45', 'V56', 'V67', 'V78', 'V89', 'sequence']:
            print('Now training: Naive Bayes {} with taxonomy level {} and hvr {} - lasted: {}'.format(k, taxonomy_level, selected_primer, time_difference_good_format(begin_time, time.time())), end='\r')
            test_size, main_class_prop, accuracy = naive_bayes_k(k=k,sequence_origin='', 
                                                                 primers_origin='DairyDB', 
                                                                 taxonomy_level=taxonomy_level,
                                                                 selected_primer=selected_primer)
            all_results.append([selected_primer, taxonomy_level, test_size, main_class_prop, accuracy])
    k_list.append(all_results)
    k_df = pd.DataFrame(k_list[k-1], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Naive Bayes - NB({})'.format(k)])
    k_df.to_csv('results/models/Naive Bayes - NB({}) - DB and GG - DB - accuracies.csv'.format(k), index=False)

In [ ]:
df_list = []
for k in range(len(k_list)):
    df_list.append(pd.DataFrame(k_list[k], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Naive Bayes - NB({})'.format(k+1)]))

final_df = pd.merge(df_list[0], df_list[1], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[2], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[3], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[4], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')

final_df.to_csv('results/models/Naive Bayes - ALL NB - DB and GG - DB.csv'.format(k), index=False)

## RF

In [ ]:
from models.random_forest import random_forest_k_default

In [ ]:
test_size, main_class_prop, accuracy = random_forest_k_default(k=4,sequence_origin='', 
                                                               primers_origin='DairyDB', 
                                                               taxonomy_level=3,
                                                               selected_primer='V4')

In [ ]:
accuracy

In [ ]:
k_list = []
begin_time = time.time()
for k in range(4,6):
    all_results = []
    for taxonomy_level in range(7):
        for selected_primer in ['V1','V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V12', 'V23', 'V34', 'V45', 'V56', 'V67', 'V78', 'V89']:
            test_size, main_class_prop, accuracy = random_forest_k_default(k=k,sequence_origin='', 
                                                                                   primers_origin='DairyDB', 
                                                                                   taxonomy_level=taxonomy_level,
                                                                                   selected_primer=selected_primer)
            all_results.append([selected_primer, taxonomy_level, test_size, main_class_prop, accuracy])
    k_list.append(all_results)
    k_df = pd.DataFrame(k_list[k-4], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Random Forest - RF({})'.format(k)])
    k_df.to_csv('results/models/Random Forest - RF({}) - DB and GG - DB - accuracies.csv'.format(k), index=False)

In [ ]:
df_list = []
for k in range(len(k_list)):
    df_list.append(pd.DataFrame(k_list[k], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Random Forest - RF({})'.format(k+4)]))

final_df = pd.merge(df_list[0], df_list[1], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
#final_df = pd.merge(final_df, df_list[2], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
#final_df = pd.merge(final_df, df_list[3], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
#final_df = pd.merge(final_df, df_list[4], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')

final_df.to_csv('results/models/Random Forest - RF 4 & 5 - DB and GG - DB.csv'.format(k), index=False)

## XGBOOST

In [ ]:
k_list = []
begin_time = time.time()
for k in range(5,0, -1):
    all_results = []
    for taxonomy_level in range(7):
        for selected_primer in ['V1','V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V12', 'V23', 'V34', 'V45', 'V56', 'V67', 'V78', 'V89', 'sequence']:
            print('Now training: XGBoost {} with taxonomy level {} and hvr {} - lasted: {}'.format(k, taxonomy_level, selected_primer, time_difference_good_format(begin_time, time.time())), end='\n')
            process = psutil.Process(os.getpid())
            print('Memory Usage before function: {:0.2f} Mb'.format(process.memory_info().rss / float(2 ** 20)), end='\n')
            test_size, main_class_prop, accuracy = xgboost_k_default(k=k,sequence_origin='DairyDB', 
                                                                     primers_origin='DairyDB', 
                                                                     taxonomy_level=taxonomy_level,
                                                                     selected_primer=selected_primer)
            process = psutil.Process(os.getpid())
            print('Memory Usage after function: {:0.2f} Mb'.format(process.memory_info().rss / float(2 ** 20)), end='\n')
            all_results.append([selected_primer, taxonomy_level, test_size, main_class_prop, accuracy])
    k_list.append(all_results)
    k_df = pd.DataFrame(k_list[5-k], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'XGBoost - XGB({})'.format(k+1)])
    k_df.to_csv('results/models/XGBoost - XGB({}) - DB - DB - accuracies.csv'.format(k), index=False)

In [ ]:
df_list = []
for k in range(len(k_list)):
    df_list.append(pd.DataFrame(k_list[k], columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'XGBoost - XGB({})'.format(5-k)]))

final_df = pd.merge(df_list[0], df_list[1], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[2], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[3], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')
final_df = pd.merge(final_df, df_list[4], on=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop'], how='left')

final_df.to_csv('results/models/XGBoost - ALL XGB - DB - DB.csv', index=False)

## XGBOOST FORCE

In [ ]:
import warnings
warnings.simplefilter('ignore')
from utils.utils import time_difference_good_format, save_update, restartkernel
import pandas as pd
import time
import os
from models.xgboost import xgboost_k_default
import time
import psutil
begin_time = time.time()
k=5
taxonomy_level=3
for selected_primer in ['sequence']:
    print('Now training: XGBoost {} with taxonomy level {} and hvr {} - lasted: {}'.format(
        k, 
        taxonomy_level, 
        selected_primer, time_difference_good_format(begin_time, time.time())), end='\n')
    test_size, main_class_prop, accuracy = xgboost_k_default(k=k,sequence_origin='DairyDB', 
                                                             primers_origin='DairyDB', 
                                                             taxonomy_level=taxonomy_level,
                                                             selected_primer=selected_primer)
    save_update('results/models/XGBoost - XGB({}) - DB - DB - accuracies.csv'.format(k), k, selected_primer, taxonomy_level, test_size, main_class_prop, accuracy)
    process = psutil.Process(os.getpid())
    print('Memory Usage: {:0.2f} Mb'.format(process.memory_info().rss / float(2 ** 20)), end='\n')
    
taxonomy_level=4
for selected_primer in ['V1','V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V12', 'V23', 'V34', 'V45', 'V56', 'V67', 'V78', 'V89', 'sequence']:
    print('Now training: XGBoost {} with taxonomy level {} and hvr {} - lasted: {}'.format(
        k, 
        taxonomy_level, 
        selected_primer, time_difference_good_format(begin_time, time.time())), end='\n')
    test_size, main_class_prop, accuracy = xgboost_k_default(k=k,sequence_origin='DairyDB', 
                                                             primers_origin='DairyDB', 
                                                             taxonomy_level=taxonomy_level,
                                                             selected_primer=selected_primer)
    save_update('results/models/XGBoost - XGB({}) - DB - DB - accuracies.csv'.format(k), k, selected_primer, taxonomy_level, test_size, main_class_prop, accuracy)
    process = psutil.Process(os.getpid())
    print('Memory Usage: {:0.2f} Mb'.format(process.memory_info().rss / float(2 ** 20)), end='\n')

Now training: XGBoost 5 with taxonomy level 3 and hvr V67 - lasted: 0 second
Memory Usage: 749.69 Mb
Now training: XGBoost 5 with taxonomy level 3 and hvr V78 - lasted: 31 minutes and 5 seconds
Memory Usage: 1328.81 Mb
Now training: XGBoost 5 with taxonomy level 3 and hvr V89 - lasted: 59 minutes and 53 seconds
Memory Usage: 1903.74 Mb
Now training: XGBoost 5 with taxonomy level 3 and hvr sequence - lasted: 1 hour, 6 minutes and 57 seconds


In [ ]:
from utils.utils import restartkernel
restartkernel()